**Introduction**

In this notebook we build a small stance classification demo using LangChain and a Hugging Face text-generation model.
Given a comment and its reply, the model should output one word: agree, disagree, or neutral.

We will:



*   Install the needed libraries.
*   Create a prompt that explains the task.
*   Load a Hugging Face model and wrap it with LangChain.
*   Run a simple chain on a few examples.
*   Improve results a bit using a few-shot prompt (showing examples in the prompt).



---


**Installing libraries**
Installing LangChain, Transformers, and the LangChain–Hugging Face bridge.

Notes:

If using a GPU, we would also need PyTorch with CUDA for your environment.

If we hit “No backend found”, we would need to install PyTorch (pip install torch --index-url https://download.pytorch.org/whl/cu121 for CUDA 12.1, or the CPU wheel).

We can safely re-run installs; pip will skip existing packages.


In [1]:
! pip install langchain
! pip install transformers
! pip install langchain-huggingface


---

**Building a reusable prompt**

Creating a PromptTemplate that takes two inputs—comment and reply—and asks the model to output only one label: agree, disagree, or neutral.


this will Keep the prompt in one place and makes it easy to reuse and reduces mistakes.

We have to be strict in prompts when we want short, clean outputs. We clearly say “output no other words”.

In [2]:
from langchain.prompts import PromptTemplate

# Define the template for stance classification
template = '''Please classify the stance, or opinion, of the following reply to the comment. Note that we want the stance of the reply to the comment, and not the stance of the reply to topic of the comment. Only give the stance as "agree", "disagree", or "neutral" and output no other words, only the label.
comment: {comment}
reply: {reply}
stance:'''

# Create the prompt object
stance_prompt = PromptTemplate(
    input_variables=["comment", "reply"],
    template=template
)

# Example usage of the prompt
comment = "Using phones right before bed makes it harder to fall asleep."
reply = "I don’t think so; I fall asleep right after scrolling."
prompt = stance_prompt.format(comment=comment, reply=reply)
print(prompt)

Please classify the stance, or opinion, of the following reply to the comment. Note that we want the stance of the reply to the comment, and not the stance of the reply to topic of the comment. Only give the stance as "agree", "disagree", or "neutral" and output no other words, only the label.
comment: Using phones right before bed makes it harder to fall asleep.
reply: I don’t think so; I fall asleep right after scrolling.
stance:




---


**Loading a Hugging Face model and wrap it for LangChain**

Creating a transformers.pipeline for text2text-generation.

and then wrapping it with HuggingFacePipeline so LangChain can call it like an LLM.

Send our stance prompt to the model and prints the raw response.

-> Make sure the model name exists on Hugging Face and we have access. If it’s private, we need to pass a token.

->If we don’t have a GPU, we need to set device=-1.

->If we see a torch/TensorFlow/Flax error, we need to install torch at minimum.

In [10]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
import torch

# we use smaller model that fits on Colab GPUs
MODEL_ID = "google/flan-t5-large"   # can try "google/flan-t5-large" if VRAM allows

hf_pipeline = pipeline(
    "text2text-generation",
    model=MODEL_ID,
    device=0 if torch.cuda.is_available() else -1,  # GPU if available, else CPU
    max_new_tokens=50,     # only need 1 word
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)
# Example usage with the prompt object from before
prompt = '''Please classify the stance, or opinion, of the following reply to the comment. Only give the stance as "agree", "disagree", or "neutral" and output no other words.
comment: Using phones right before bed makes it harder to fall asleep.
reply: I agree,once I stopped scrolling at night, I slept faster.
stance:'''

# Get the model's response
response = llm(prompt)
print(response)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


agree




---


**Creating a chain: prompt → model**

We will Build a simple chain using the pipe operator (|).

stance_prompt | llm : formats the prompt with inputs, then send it to the model.

We will run .invoke({...}) with a comment and reply and print the result.

->Chains are clean and composable. We can later add more steps (e.g., parsing, validation) without changing the rest, that's why it is useful

In [11]:
stance_chain = stance_prompt | llm

# Example usage: run the chain with the provided comment and reply
comment = "Taking a 20-minute walk each day improves mood and energy."
reply = "I agree, even a short walk makes me feel more alert."

# Format the input and get the result
result = stance_chain.invoke({"comment": comment, "reply": reply})

print(result)

agree




---
**Preparing the test data**

We will define one fixed comment and a small list of replies with carying stances.

This will give us a quick way to see how the model behaves with different kinds of replies: on-topic, off-topic, and noisy.

In [12]:
test_comment = "Using phones right before bed makes it harder to fall asleep."

test_replies = [
    "Totally—my mind keeps racing after browsing.",
    "I’m fine; I can sleep in minutes even after watching videos.",
    "Maybe, but only if the room is bright.",
    "Once I put my phone away an hour before bed, I slept better.",
    "Nah, dark mode and low brightness solve it.",
    "Hard to say; caffeine and stress affect me more.",
    "Lol I scroll every night and still pass out.",
    "Blue light filters help a bit, but not always."
]





---


**Running the chain on all test replies**

We will loop through every reply, call the chain, store the outputs, and print a neat summary.

-> We must check if the model return only one of agree / disagree / neutral?

-> Does the model stay on task when the reply is off-topic?


In [13]:
responses = []
for reply in test_replies:
    response = stance_chain.invoke({"comment": test_comment, "reply": reply})
    responses.append(response)

# Print the results
for idx, (reply, response) in enumerate(zip(test_replies, responses)):
    print(f"Reply {idx+1}: {reply}\nStance: {response}\n")

Reply 1: Totally—my mind keeps racing after browsing.
Stance: agree

Reply 2: I’m fine; I can sleep in minutes even after watching videos.
Stance: agree

Reply 3: Maybe, but only if the room is bright.
Stance: agree

Reply 4: Once I put my phone away an hour before bed, I slept better.
Stance: agree

Reply 5: Nah, dark mode and low brightness solve it.
Stance: disagree

Reply 6: Hard to say; caffeine and stress affect me more.
Stance: neutral

Reply 7: Lol I scroll every night and still pass out.
Stance: agree

Reply 8: Blue light filters help a bit, but not always.
Stance: neutral





---


**Building a few-shot prompt**

Creating a FewShotPromptTemplate with several labeled examples.

The prefix will explain the task.

The examples will show the desired input/output format.

The suffix will hold the final query (our actual comment and reply).


In [14]:
from langchain.prompts import FewShotPromptTemplate

# Define the prompt template for each example
example_template = '''comment: {comment}
reply: {reply}
stance: {stance}'''

example_prompt = PromptTemplate(
    input_variables=["comment", "reply", "stance"],
    template=example_template
)

# Define the examples with various stances
examples = [
    {
        'comment': "Doing a 5-minute stretch each morning improves flexibility.",
        'reply': "Yes, I feel looser after a week of daily stretches.",
        'stance': 'agree'
    },
    {
        'comment': "Listening to music while studying improves concentration.",
        'reply': "Not for me—lyrics pull my attention away.",
        'stance': 'disagree'
    },
    {
        'comment': "Drinking eight glasses of water a day reduces headaches.",
        'reply': "It might help some people, but not always for me.",
        'stance': 'neutral'
    },
    {
        'comment': "Eating breakfast helps you focus better for the rest of the morning.",
        'reply': "Sometimes, but if I eat heavy I feel sleepy.",
        'stance': 'neutral'
    },
    {
        'comment': "Keeping houseplants in a room improves indoor air quality.",
        'reply': "I doubt it; my allergies got worse with plants around.",
        'stance': 'disagree'
    },
    {
        'comment': "A 20-minute afternoon nap boosts memory.",
        'reply': "Yes, I recall details better after short naps.",
        'stance': 'agree'
    },
    {
        'comment': "Using checklists reduces mistakes in daily tasks.",
        'reply': "Absolutely—my to-do list catches things I forget.",
        'stance': 'agree'
    },
    {
        'comment': "Playing puzzle games every day sharpens attention.",
        'reply': "I don’t think so; I saw no change after months of puzzles.",
        'stance': 'disagree'
    }
]

# Define the prefix and suffix for the few-shot prompt
prefix = '''Stance classification is the task of determining the expressed or implied opinion, or stance, of a reply toward a comment. The following replies express opinions about the associated comment. Each reply can either be "agree", "disagree", or "neutral" toward the comment.'''

suffix = '''Analyze the following reply to the provided comment and determine its stance. Respond with a single word: "agree", "disagree", or "neutral". Only return the stance as a single word, and no other text.
comment: {comment}
reply: {reply}
stance:'''

# Create the FewShotPromptTemplate using the provided prefix, suffix, and examples
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["comment", "reply"],
    example_separator="\n"
)




---


**Run the few-shot chain on the same test set**

We will Pipe the few-shot prompt into the same llm.

Run the test replies again and print the stances.

What we are checking here is :

->Are the labels more stable?

->Did off-topic replies become more often neutral?

->Are there fewer extra words in the output?


In [15]:
few_shot_chain = few_shot_prompt | llm

responses = []
for reply in test_replies:
    response = few_shot_chain.invoke({"comment": test_comment, "reply": reply})
    responses.append(response)
    print(responses)

# Print the results
for idx, (reply, response) in enumerate(zip(test_replies, responses)):
    print(f"Reply {idx+1}: {reply}\nStance: {response}\n")

['agree']
['agree', 'disagree']
['agree', 'disagree', 'disagree']
['agree', 'disagree', 'disagree', 'agree']
['agree', 'disagree', 'disagree', 'agree', 'disagree']
['agree', 'disagree', 'disagree', 'agree', 'disagree', 'neutral']
['agree', 'disagree', 'disagree', 'agree', 'disagree', 'neutral', 'agree']
['agree', 'disagree', 'disagree', 'agree', 'disagree', 'neutral', 'agree', 'agree']
Reply 1: Totally—my mind keeps racing after browsing.
Stance: agree

Reply 2: I’m fine; I can sleep in minutes even after watching videos.
Stance: disagree

Reply 3: Maybe, but only if the room is bright.
Stance: disagree

Reply 4: Once I put my phone away an hour before bed, I slept better.
Stance: agree

Reply 5: Nah, dark mode and low brightness solve it.
Stance: disagree

Reply 6: Hard to say; caffeine and stress affect me more.
Stance: neutral

Reply 7: Lol I scroll every night and still pass out.
Stance: agree

Reply 8: Blue light filters help a bit, but not always.
Stance: agree

